# Generating a Subckt to the Chip Pins to "PCB" RBUS and SWBUS Nodes in the Schematic

In [4]:
import csv
import json
import os
from datetime import datetime

In [5]:
# Change the working directory to the script's directory

working_dir = "/Users/peterkinget/iCloudDrive/Work/MOSBIUS/MOSbiusTools/MOSbiusCADFlow/MOSbiusV2Tools/MOSbiusV2Tools/tmp/"
os.chdir(working_dir)

# # Define the file path
# file_path = "MOSbiusV2_sw_matrix_register_map_20250503.csv"

# # Check if the script is being run from the correct directory
# if not os.path.exists(file_path):
#     print(f"Error: The script must be run from the directory containing {filename}.")
#     sys.exit(1) 

## RBUS + SBUS direct

In [35]:
def generate_circuit_file(circuit_file, pin_mapping_file, template_file, output_file):
    """
    Generates a SPICE subcircuit file based on the circuit JSON file and pin mapping.

    Args:
        circuit_file (str): Path to the circuit JSON file (e.g., RO3_circuit.json).
        pin_mapping_file (str): Path to the pin_name_to_number.json file.
        template_file (str): Path to the SPICE template file.
        output_file (str): Path to the output SPICE file.
    """
    # Load the circuit JSON file
    with open(circuit_file, 'r') as f:
        circuit_data = json.load(f)

    # Load the pin mapping JSON file
    with open(pin_mapping_file, 'r') as f:
        pin_mapping = json.load(f)

    # Read the SPICE template
    with open(template_file, 'r') as f:
        spice_template = f.read()

    # Start building the SPICE subcircuit
    spice_subcircuit = f"* File created on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
    spice_subcircuit += f"* From {circuit_file}\n" + spice_template + "\n"

    # Iterate over each BUS in the circuit data
    for bus, pins in circuit_data.items():
        if bus[0:4] == 'SBUS':
            continue
        # Replace RBUS1 with RBUS<1>, RBUS2 with RBUS<2>, etc.
        bus_with_brackets = bus.replace("RBUS", "RBUS<").replace("SWBUS", "SWBUS<") + ">"

        # Select the first pin from the list of connected pins
        selected_pin = pins[0]

        # Get the pin number from the pin mapping
        pin_number = int(pin_mapping[selected_pin])

        # Add a zero-volt voltage source for the connection
        spice_subcircuit += f"V{bus}_to_pin{pin_number} {bus_with_brackets} pin<{pin_number}> 0\n"

        # Add a comment indicating the connection
        spice_subcircuit += f"* {bus} connected to {selected_pin} (pin<{pin_number}>)\n"

    for sbus in range(1,7):
        if sbus == 6:
            bus = f"DATA_SBUS6"
        else:
            bus = f"SBUS{sbus}"
        bus_with_brackets = f"SWBUS<{sbus}>"
        pin_number = int(pin_mapping[bus])
        # Add a zero-volt voltage source for the connection
        spice_subcircuit += f"V{bus}_to_pin{pin_number} {bus_with_brackets} pin<{pin_number}> 0\n"

    # Close the subcircuit in spice
    spice_subcircuit += ".ENDS \n"
    
    # Write the generated SPICE subcircuit to the output file
    with open(output_file, 'w') as f:
        f.write(spice_subcircuit)

In [36]:
# Example usage
generate_circuit_file(
    circuit_file="../../examples/INV_string_clocked_RBUS_SBUS.json",
    pin_mapping_file="pin_name_to_number.json",
    template_file="PK_pins_to_RBUS_SWBUS_template.cir",
    output_file="generated_pins_to_RBUS_SBUS_2.cir"
)


## RBUS and SBUS --> not necessary ... direct pins for SBUS available!!

In [27]:
def generate_circuit_file_2(circuit_file, pin_mapping_file, template_file, output_file):
    """
    Generates a SPICE subcircuit file based on the circuit JSON file and pin mapping.

    Args:
        circuit_file (str): Path to the circuit JSON file (e.g., RO3_circuit.json).
        pin_mapping_file (str): Path to the pin_name_to_number.json file.
        template_file (str): Path to the SPICE template file.
        output_file (str): Path to the output SPICE file.
    """
    # Load the circuit JSON file
    with open(circuit_file, 'r') as f:
        circuit_data = json.load(f)

    # Load the pin mapping JSON file
    with open(pin_mapping_file, 'r') as f:
        pin_mapping = json.load(f)

    # Read the SPICE template
    with open(template_file, 'r') as f:
        spice_template = f.read()

    # Start building the SPICE subcircuit
    spice_subcircuit = f"* File created on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
    spice_subcircuit += f"* From {circuit_file}\n" + spice_template + "\n"

    # Iterate over each BUS in the circuit data
    for bus, pins in circuit_data.items():
        # Replace RBUS1 with RBUS<1>, RBUS2 with RBUS<2>, etc.
        bus_with_brackets = bus.replace("RBUS", "RBUS<").replace("SBUS", "SWBUS<") + ">"

        # Select the first pin from the list of connected pins
        if bus[0:4] == 'RBUS':
            selected_pin = pins[0]
        elif bus[0:4] == 'SBUS':
            selected_pin = pins[0]['terminal']
        # Get the pin number from the pin mapping
        pin_number = int(pin_mapping[selected_pin])

        # Add a comment indicating the connection
        spice_subcircuit += f"* {bus} connected to {selected_pin} (pin<{pin_number}>)\n"

        # Add a zero-volt voltage source for the connection
        spice_subcircuit += f"V{bus}_to_pin{pin_number} {bus_with_brackets} pin<{pin_number}> 0\n"

        
    # Close the subcircuit in spice
    spice_subcircuit += ".ENDS \n"
    
    # Write the generated SPICE subcircuit to the output file
    with open(output_file, 'w') as f:
        f.write(spice_subcircuit)

In [30]:
generate_circuit_file_2(
    circuit_file="../../examples/INV_string_11_CC_RBUS_SBUS.json",
    pin_mapping_file="pin_name_to_number.json",
    template_file="PK_pins_to_RBUS_SWBUS_template.cir",
    output_file="generated_pins_to_RBUS_SBUS.cir"
)
